In [3]:
import pandas as pd
df=pd.read_csv('cars_data.csv')
print(df.head())

   listing_id                                                url  \
0      121221  https://www.automobile.tn/fr/occasion/mercedes...   
1      119831  https://www.automobile.tn/fr/occasion/jaguar/f...   
2      120005  https://www.automobile.tn/fr/occasion/smart/fo...   
3      117487  https://www.automobile.tn/fr/occasion/mercedes...   
4      115361  https://www.automobile.tn/fr/occasion/mercedes...   

           brand           model     version  mileage  year transmission  \
0  Mercedes-Benz          CLAAMG   Edition 1     3653  2025  Automatique   
1         Jaguar  F-TypeRDYNAMIC         NaN    12000  2017  Automatique   
2          Smart   ForTwoPassion         NaN    44000  2000  Automatique   
3  Mercedes-Benz             CLE  CoupéAMG +    15000  2024  Automatique   
4  Mercedes-Benz          Classe  SLimousine   212000  2008  Automatique   

   drivetrain fuel_type  ...  previous_owners body_type   exterior_color  \
0    Traction   Hybride  ...        1ère main   Berline  G

In [ ]:
df.drop